In [1]:
import json
import os
import time
import loguru
import requests
import webbrowser
import sinaweibopy3
from argparse import Namespace


logger = loguru.logger

headers = {
    'Referer': 'http:www.baidu.com',
    "Upgrade-Insecure-Requests": "1",
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36',
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8,la;q=0.7,pl;q=0.6",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8"
}


app1=Namespace(
    APP_KEY = '3197544314',
    APP_SECRET = '4413b78f879d08c0fd1afd69a9bc976c',
    REDIRECT_URL = 'https://api.weibo.com/oauth2/default.html'
)

app2=Namespace(
    APP_KEY = '3621364603',
    APP_SECRET = 'a8a1944bd8cca5624c30c371d7ce0e8b',
    REDIRECT_URL = 'https://api.weibo.com/oauth2/default.html'
)


/home/jasonhaven/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# 访问授权有频次限制
try:
    client = sinaweibopy3.APIClient(app_key=app2.APP_KEY, app_secret=app2.APP_SECRET, redirect_uri=app2.REDIRECT_URL)
    url = client.get_authorize_url()
    logger.debug(url)
    webbrowser.open_new(url)
    
    result = client.request_access_token(input("please input code : "))  # Enter the CODE obtained in the authorized address
    logger.debug(result)
    
    # 过期时间 = 用户授权时间 + 授权有效期
    logger.info('expires in {}'.format(time.asctime( time.localtime(time.time()+result.expires_in))))
    access_token=result.access_token
    client.set_access_token(result.access_token, result.expires_in)

except ValueError:
    logger.debug('pyOauth2Error')

2019-04-25 22:51:24.080 | DEBUG    | __main__:<module>:5 - https://api.weibo.com/oauth2/authorize?response_type=code&client_id=3621364603&redirect_uri=https://api.weibo.com/oauth2/default.html
2019-04-25 22:51:38.526 | DEBUG    | __main__:<module>:9 - {'access_token': '2.009GpigHV3sExD568780f376gd9U3C', 'remind_in': '157679999', 'expires_in': 1713883897, 'uid': '7044218812', 'isRealName': 'true'}
2019-04-25 22:51:38.527 | INFO     | __main__:<module>:12 - expires in Wed Aug 16 13:43:15 2073


In [3]:
def current_user_timeline(uid=7044218812,count=20,page=1,feature=0,trim_user=0):
    '''
    #获取当前用户最新发表的微博列表
    == current_user_show
    count:默认20,最大100
    page:>=1
    feature:过滤类型ID，0：全部、1：原创、2：图片、3：视频、4：音乐，默认为0。
    trim_user:返回值中user字段开关，0：返回完整user字段、1：user字段仅返回user_id，默认为0。
    '''
    params={}
    params['access_token']=access_token
    params['uid']=uid
    params['count']= count 
    params['page']=page 
    params['feature']=feature
    url='https://api.weibo.com/2/statuses/user_timeline.json'
    rsp=requests.get(url,params=params,headers=headers)
    logger.debug(rsp.url)
    data=json.loads(rsp.text)
    return data

def home_timeline(count=20,page=1,feature=0,trim_user=0):
    '''
    #获取当前登录用户及其所关注（授权）用户的最新微博
    count:默认20,最大100
    page:>=1
    feature:过滤类型ID，0：全部、1：原创、2：图片、3：视频、4：音乐，默认为0。
    trim_user:返回值中user字段开关，0：返回完整user字段、1：user字段仅返回user_id，默认为0。
    '''
    params={}
    params['access_token']=access_token
    params['count']= count 
    params['page']=page 
    params['feature']=feature
    url='https://api.weibo.com/2/statuses/home_timeline.json'
    rsp=requests.get(url,params=params,headers=headers)
    logger.debug(rsp.url)
    data=json.loads(rsp.text)
    return data

def users_counts(uids:list):
    '''
    #获取用户的粉丝数、关注数、微博数
    '''
    params={}
    params['access_token']=access_token
    params['uids']=','.join(uids)
    url='https://api.weibo.com/2/users/counts.json'
    rsp=requests.get(url,params=params,headers=headers)
    logger.debug(rsp.url)
    data=json.loads(rsp.text)
    return data

def comments_show(sid:int,count=20,page=1,filter_by_author=0):
    '''
    #根据微博id获取评论列表
    count:单页返回的记录条数，默认为50。
    page:默认为1
    filter_by_author:作者筛选类型，0：全部、1：我关注的人、2：陌生人，默认为0。
    '''
    params={}
    params['access_token']=access_token
    params['id']=sid
    params['count']=count
    params['page']=page
    params['filter_by_author']=filter_by_author
    url='https://api.weibo.com/2/comments/show.json'
    rsp=requests.get(url,params=params,headers=headers)
    logger.debug(rsp.url)
    data=json.loads(rsp.text)
    return data

def parse_timeline(data):
    '''
    #解析微博列表数据
    '''
    result=[]
    if 'error' in data.keys():
        logger.error(data)
    else:
        result=[]
        for s in data['statuses']:
            dct={'status':{},'user':s['user']}
            dct['status']['created_at'] = s['created_at']
            dct['status']['id'] = s['id']
            dct['status']['text'] = s['text']
            dct['status']['reposts_count'] = s['reposts_count']
            dct['status']['comments_count'] = s['comments_count']
            dct['status']['attitudes_count'] = s['attitudes_count']
            dct['status']['isLongText']=s['isLongText']
            result.append(dct)
    return result

def parse_comments(data):
    '''
    #解析评论数据
    '''
    result=[]
    if 'error' in data.keys():
        logger.error(data)
    else:
        result=[]
        for s in data['comments']:
            dct={'comment':{},'user':s['user']}
            dct['comment']['created_at'] = s['created_at']
            dct['comment']['id'] = s['id']
            dct['comment']['text'] = s['text']
            result.append(dct)
    return result

def status_by_current_user_sid(sid:int):
    '''
    #微博ID获取单条微博内容,但是查询的微博必须是授权用户发出
    '''
    params={}
    params['access_token']=access_token
    params['id']=sid
    url = 'https://api.weibo.com/2/statuses/show.json'
    rsp=requests.get(url,params=params,headers=headers)
    logger.debug(rsp.url)
    data=json.loads(rsp.text)
    return data

In [5]:
comments_show?

Signature: comments_show(sid:int, count=20, page=1, filter_by_author=0)
Docstring:
#根据微博id获取评论列表
count:单页返回的记录条数，默认为50。
page:默认为1
filter_by_author:作者筛选类型，0：全部、1：我关注的人、2：陌生人，默认为0。
File:      /media/jasonhaven/code/weibo/src/<ipython-input-3-6b7b2c44de5e>
Type:      function


In [6]:
comments_show(4364347108579326)

2019-04-25 22:54:25.508 | DEBUG    | __main__:comments_show:69 - https://api.weibo.com/2/comments/show.json?access_token=2.009GpigHV3sExD568780f376gd9U3C&id=4364347108579326&count=20&page=1&filter_by_author=0


{'comments': [{'created_at': 'Thu Apr 25 14:54:17 +0800 2019',
   'id': 4364977756326292,
   'rootid': 4364977756326292,
   'floor_number': 282,
   'text': 'superga中国首位品牌大使，不声不响做大事。林彦俊棒',
   'disable_reply': 0,
   'user': {'id': 6535824152,
    'idstr': '6535824152',
    'class': 1,
    'screen_name': '八先生的八夫人',
    'name': '八先生的八夫人',
    'province': '100',
    'city': '1000',
    'location': '其他',
    'description': '吃肉肉长肉肉，不吃肉肉精瘦瘦',
    'url': '',
    'profile_image_url': 'http://tvax4.sinaimg.cn/crop.0.0.664.664.50/0078jEqAly8g1qos3r867j30ig0igabd.jpg',
    'cover_image_phone': 'http://ww1.sinaimg.cn/crop.0.0.640.640.640/9d44112bjw1f1xl1c10tuj20hs0hs0tw.jpg',
    'profile_url': 'u/6535824152',
    'domain': '',
    'weihao': '',
    'gender': 'f',
    'followers_count': 39,
    'friends_count': 142,
    'pagefriends_count': 9,
    'statuses_count': 1243,
    'video_status_count': 0,
    'favourites_count': 15,
    'created_at': 'Thu Apr 26 16:41:14 +0800 2018',
    'following': Fals

In [19]:
users_counts(['1642591402'])

2019-04-20 15:16:08.188 | DEBUG    | __main__:users_counts:50 - https://api.weibo.com/2/users/counts.json?access_token=2.009GpigHV3sExD568780f376gd9U3C&uids=1642591402


[{'id': 1642591402,
  'followers_count': 26656868,
  'friends_count': 3376,
  'statuses_count': 138037,
  'private_friends_count': 0,
  'pagefriends_count': 1651}]

In [6]:
def following_list(uid,count=5,trim_status=1,cursor=0):
    '''
    # 10023：用户请求超过上限
    #获取用户的关注列表
    
    count:单页返回的记录条数，默认为5，最大不超过5。
    trim_status:返回值中user字段中的status字段开关，0：返回完整status字段、1：status字段仅返回status_id，默认为1。
    cursor:回结果的游标，下一页用返回值里的next_cursor，上一页用previous_cursor，默认为0。
    '''
    params={}
    params['access_token']=access_token
    params['uid']=uid
    params['count']=count
    params['trim_status']=trim_status
    params['cursor']=cursor
    url='https://api.weibo.com/2/friendships/friends/ids.json'
    rsp=requests.get(url,params=params,headers=headers)
    logger.debug(rsp.url)
    data=json.loads(rsp.text)
    return data,data['next_cursor']

In [9]:
next_cursor=0
data,next_cursor=following_list(7044218812,cursor=next_cursor)
print(data)

2019-04-20 14:58:55.700 | DEBUG    | __main__:following_list:18 - https://api.weibo.com/2/friendships/friends/ids.json?access_token=2.009GpigHV3sExD568780f376gd9U3C&uid=7044218812&count=5&trim_status=1&cursor=0


{'ids': [1255795640], 'next_cursor': 5, 'previous_cursor': 0, 'total_number': 97}


In [15]:
data,next_cursor=following_list(7044218812,cursor=next_cursor)
print(data)

2019-04-20 14:59:27.464 | DEBUG    | __main__:following_list:18 - https://api.weibo.com/2/friendships/friends/ids.json?access_token=2.009GpigHV3sExD568780f376gd9U3C&uid=7044218812&count=5&trim_status=1&cursor=0


{'ids': [1255795640], 'next_cursor': 5, 'previous_cursor': 0, 'total_number': 97}


In [240]:
# current_user_show()
# current_user_timeline()
# users_counts(['7044218812','5909342713','3623353053'])


In [242]:
# statuses = home_timeline()
# statuses = parse_timeline(statuses)

In [243]:
# print(statuses[0]['status'])
# comments = comments_show(statuses[0]['status']['id'],count=10)

In [244]:
# status_by_sid(4362796038844430)